In [1]:
import time
import csv
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import optimizers
import tensorflow.keras.backend as K
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
import os
import pandas as pd
from sklearn.preprocessing import OneHotEncoder



In [2]:
#""" Loading, padding and one-hot encoding of the data"""

url = 'https://raw.githubusercontent.com/dbosnacki/HelisDeepLearningCourse/main/cath-domain-description-file-v2_4ProcessedForNN.tsv' 
df = pd.read_csv(url, delimiter = "\t", header=None)
npd = df[3].to_numpy()



In [3]:
# find the maximal sequence length
maxSeqLength = 0
for sequence in npd:
    l = len(sequence)
    if l > maxSeqLength:
      maxSeqLength = l 

data = list()
for i in range(len(npd)):
  alph_as_num = np.array(list(npd[i])).view(np.int32)
  data.append(np.array(list(npd[i])).view(np.int32))

#print(len(data))
#print(data[1330])

padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(
    data, padding="post"
)
#print(padded_inputs)

In [4]:
# padd the sequences with spaces to get equal length
# dataset = []

# for sequence in sequences:
#     dataset.append(list(sequence.ljust(maxSeqLength, ' ')))
    
# #one hot encoding of the data
# cat = OneHotEncoder()
# dataset = cat.fit_transform(dataset).toarray()

labels = list(df[2])    

X = padded_inputs
y = np.array(labels)

print(X.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

accuracy_per_fold = []
loss_per_fold = []

fold_no = 1
seed = 10
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)



(94785, 70)


In [ ]:
for train_index, val_index in skf.split(X_train, y_train):
    
    print('Fold: ' + str(fold_no))
    

    # Define RNN model 
    model = keras.Sequential(
        [
           layers.Embedding(128, 8),
           layers.LSTM(64, name="rnn1"),
           #layers.Dense(64, activation="relu", name="layer2"),
           layers.Dense(3, activation="softmax", name="output"),
        ]
    )
    
    # model.summary()

    model.compile(
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=['accuracy'],
        optimizer='adam',
    )
    

    history = model.fit(X_train[train_index], 
                        y_train[train_index], 
                        batch_size = 128, 
                        epochs = 100, 
                        #class_weight = class_weight, 
                        validation_data = (X_train[val_index], y_train[val_index]),
                        #callbacks = callbacks_list,
                        verbose = 2)
    
    scores = model.evaluate(X_test, y_test, verbose=2)
    
    #model.save(newpath + r'\fold-' + str(fold_no) + '.hdf5') 

    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    accuracy_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    # Increase fold number
    fold_no = fold_no + 1

# Average scores
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(len(accuracy_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {accuracy_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(accuracy_per_fold)} (+- {np.std(accuracy_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

Fold: 2
Epoch 1/100
474/474 - 3s - loss: 0.9461 - accuracy: 0.5693 - val_loss: 0.8606 - val_accuracy: 0.6083
Epoch 2/100
474/474 - 3s - loss: 0.8556 - accuracy: 0.6039 - val_loss: 0.8481 - val_accuracy: 0.6064
Epoch 3/100
474/474 - 3s - loss: 0.8423 - accuracy: 0.6098 - val_loss: 0.8444 - val_accuracy: 0.6186
Epoch 4/100
474/474 - 3s - loss: 0.8265 - accuracy: 0.6202 - val_loss: 0.8102 - val_accuracy: 0.6357
Epoch 5/100
474/474 - 3s - loss: 0.8084 - accuracy: 0.6354 - val_loss: 0.7988 - val_accuracy: 0.6394
Epoch 6/100
474/474 - 3s - loss: 0.7930 - accuracy: 0.6433 - val_loss: 0.7846 - val_accuracy: 0.6466
Epoch 7/100
474/474 - 3s - loss: 0.7742 - accuracy: 0.6547 - val_loss: 0.7577 - val_accuracy: 0.6637
Epoch 8/100
474/474 - 3s - loss: 0.7543 - accuracy: 0.6664 - val_loss: 0.7408 - val_accuracy: 0.6782
Epoch 9/100
474/474 - 3s - loss: 0.7315 - accuracy: 0.6848 - val_loss: 0.7199 - val_accuracy: 0.6925
Epoch 10/100
474/474 - 3s - loss: 0.7161 - accuracy: 0.6941 - val_loss: 0.7158 - va